# Question 5

In [435]:
import numpy as np
import pandas as pd
import igraph
from igraph import *

In [3]:
movie_graph = read("movie_pairs.csv",format = "ncol", weights = True, directed = False)
#movie_graph.es.attribute_names()

In [4]:
%%time
communities = movie_graph.community_fastgreedy(weights=movie_graph.es['weight'])

CPU times: user 3h 48min 41s, sys: 26 s, total: 3h 49min 7s
Wall time: 3h 49min 11s


In [436]:
movie_count = pd.read_csv("movie_count.csv",header = None,names=["movie","count","id"])

In [437]:
movie_count.head(3)

,movie,count,id
0,10 Attitudes (2001),12,0
1,101 Reykjav�k (2000),11,1
2,12 Years a Slave (2013) (uncredited),20,2


In [440]:
movie_genre = pd.read_csv("movie_genre.txt",sep = "\t",header = None, names= ["movie","1","genre"])
movie_genre = movie_genre.drop("1",axis=1)

In [444]:
print len(movie_count)
print len(movie_genre)

98749
1010991


In [55]:
communities.optimal_count # number of communities

24

In [453]:
clusters = communities.as_clustering()
comm_membership = clusters.membership
total_clusters=np.unique(comm_membership)
print total_clusters

movie_vertices_id=movie_graph.vs['name']
vertex_id_membership=pd.DataFrame({"id":movie_vertices_id,"membership":comm_membership})
vertex_id_membership.id = vertex_id_membership.id.astype(np.int64)
vertex_id_membership.head(5)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]


,id,membership
0,4,0
1,17,0
2,13,1
3,19,1
4,7,2


In [452]:
movie_count.head(5)

,movie,count,id
0,10 Attitudes (2001),12,0
1,101 Reykjav�k (2000),11,1
2,12 Years a Slave (2013) (uncredited),20,2
3,22 Jump Street (2014),22,3
4,45 Fathers (1937) (uncredited),29,4


In [454]:
movie_id_membership = movie_count.join(vertex_id_membership.set_index("id"),on="id",how = "inner")

In [455]:
movie_id_membership.head(4)

,movie,count,id,membership
0,10 Attitudes (2001),12,0,6
1,101 Reykjav�k (2000),11,1,5
2,12 Years a Slave (2013) (uncredited),20,2,6
3,22 Jump Street (2014),22,3,6


In [245]:
len(movie_id_membership)

98748

In [121]:
movie_genre.head(4)

,movie,genre
0,Hearts of Iron (2002),Adventure
1,Behind Stone Walls (1932),Romance
2,Three Doors Down and Daughtry Live (2012),Music
3,Super Seniors (2008),Short


In [466]:
movie_genre["Newname"]=movie_genre["movie"].map(lambda x:x.decode("unicode_escape").encode("ascii","ignore"))
#movie_genre.head(100)

In [456]:
id_genre = movie_id_membership.merge(movie_genre, left_on="movie", right_on="movie",how = "left")

In [468]:
def movie_subname(name):
    newname=name.decode("unicode_escape").encode("ascii","ignore")
    end=newname.find('(uncredited)')
    if (end>0):
        return newname[0:end].strip()
    else:
        return newname
def movie_subname2(name):
    newname=name.decode("unicode_escape").encode("ascii","ignore")
    firstright=newname.find(')')
    return newname[0:(firstright+1)].strip()

movie_id_membership["movie"]=movie_id_membership["movie"].astype('str')
#movie_id_membership["Newname"]=movie_id_membership["movie"].map(lambda x:movie_subname(x))
movie_id_membership["Newname2"]=movie_id_membership["movie"].map(lambda x:movie_subname2(x))

In [471]:
id_genre1 = movie_id_membership.merge(movie_genre, left_on="Newname2", right_on="Newname",how = "left")

In [473]:
id_genre1[pd.isnull(id_genre1["genre"])==True]

,movie_x,count,id,membership,Newname2,movie_y,genre,Newname
12,Ai no rekishi (1955),11,12,1,Ai no rekishi (1955),NaN,NaN,NaN
13,Aijin (1953),11,13,1,Aijin (1953),NaN,NaN,NaN
18,Ama - Nangoku-hen (1926),11,18,1,Ama - Nangoku-hen (1926),NaN,NaN,NaN
19,Ame no naka ni kiete (1963),15,19,1,Ame no naka ni kiete (1963),NaN,NaN,NaN
22,Anino ni Sisa (1968),11,22,3,Anino ni Sisa (1968),NaN,NaN,NaN
47,Bob und Mary (1923),12,47,9,Bob und Mary (1923),NaN,NaN,NaN
49,Bolidi sull'asfalto a tutta birra! (1970),10,49,5,Bolidi sull'asfalto a tutta birra! (1970),NaN,NaN,NaN
67,Clara et les m�chants (1958),10,67,5,Clara et les mchants (1958),NaN,NaN,NaN
83,Cs�k� �s kalap (1941),22,83,8,Csk s kalap (1941),NaN,NaN,NaN
97,Der Roman eines Dienstm�dchens (1921),17,97,9,Der Roman eines Dienstmdchens (1921),NaN,NaN,NaN


In [474]:
id_genre = id_genre1[["membership","genre"]]

In [475]:
id_genre = id_genre.fillna("unknown")

In [479]:
id_genre["number"] = 1

In [482]:
genre_membership=pd.pivot_table(id_genre,index="membership",columns= "genre",values="number",aggfunc=np.sum)

In [484]:
genre_membership = genre_membership.fillna(0)
genre_membership["sum"] = genre_membership.sum(axis=1)

In [491]:
#id_genre_new.groupby("membership")["genre"].apply(list)

In [509]:
genre_membership2 = genre_membership.apply(lambda row: row[:-1]/row["sum"],axis=1)

In [647]:
labels=genre_membership2.apply(lambda row:np.where(row>0.15),axis=1).map(lambda x:x[0])

In [648]:
genres=list(genre_membership2.columns)
genres=np.asarray(genres)
types=[genres[x].tolist() if len(x)>0 else [] for x in labels]

In [649]:
types

[['Drama', 'Romance', 'Western'],
 ['Drama', 'unknown'],
 ['Drama'],
 ['Action', 'Comedy', 'Drama'],
 ['Drama', 'Romance'],
 ['Comedy', 'Drama'],
 ['Drama', 'Thriller'],
 ['Comedy', 'Drama', 'Romance'],
 ['Comedy', 'Drama', 'unknown'],
 ['Comedy', 'Drama', 'unknown'],
 ['Drama', 'Romance', 'unknown'],
 ['Comedy', 'Drama'],
 ['Comedy', 'Drama'],
 ['Drama'],
 ['Drama', 'Romance'],
 ['Drama', 'War'],
 ['Short'],
 ['Comedy', 'Drama', 'Romance'],
 ['Comedy', 'Drama'],
 ['Drama', 'Romance', 'Short', 'Thriller'],
 ['Comedy', 'Drama', 'unknown'],
 ['Drama', 'War'],
 ['Drama', 'unknown'],
 ['Drama', 'Romance']]

In [650]:
genre_clustrer = pd.DataFrame({"community":total_clusters,"genre":types,"community_size":genre_membership["sum"].tolist()})

In [651]:
genre_clustrer

,community,community_size,genre
0,0,18293.0,"[Drama, Romance, Western]"
1,1,4636.0,"[Drama, unknown]"
2,2,3689.0,[Drama]
3,3,2912.0,"[Action, Comedy, Drama]"
4,4,1196.0,"[Drama, Romance]"
5,5,15966.0,"[Comedy, Drama]"
6,6,24781.0,"[Drama, Thriller]"
7,7,1183.0,"[Comedy, Drama, Romance]"
8,8,5204.0,"[Comedy, Drama, unknown]"
9,9,5655.0,"[Comedy, Drama, unknown]"


In [404]:
id_genre.to_csv("id_genre.csv",header=True,index=False)
vertex_id_membership.to_csv("vertex_id_membership.csv",header=True,index=False)

# Question 6

In [626]:
id_member_movie = movie_id_membership[["id","membership","movie"]]
movie_genre1 = pd.read_csv("movie_genre.txt",sep = "\t",header = None, names= ["movie","1","genre"])
movie_genre1 = movie_genre1.drop("1",axis=1)

In [535]:
movie_pairs = pd.read_csv("movie_pairs.csv", sep=" ",header = None, names=["movie1","movie2","weight"])
movie_count = pd.read_csv("movie_count.csv",header = None,names=["movie","count","id"])

In [627]:
def movie_subname3(name):
    firstright=name.find(')')
    return name[0:(firstright+1)].strip()
#movie_count["movie"]=movie_count["movie"].astype('str')
#movie_count["movie"]=movie_count["movie"].map(lambda x:movie_subname3(x))
#movie_pairs["movie1"]=movie_pairs["movie1"].astype('str')
#movie_pairs["movie2"]=movie_pairs["movie2"].astype('str')
#movie_pairs["movie1"]=movie_pairs["movie1"].map(lambda x:movie_subname3(x))
#movie_pairs["movie2"]=movie_pairs["movie2"].map(lambda x:movie_subname3(x))

### Batman v Superman: Dawn of Justice (2016)

In [536]:
movie_count[movie_count["movie"]=="Batman v Superman: Dawn of Justice (2016)"]

,movie,count,id
9913,Batman v Superman: Dawn of Justice (2016),54,171798691881


In [621]:
id_member_movie[id_member_movie["id"] == 171798691881]

,id,membership,movie
9913,171798691881,6,Batman v Superman: Dawn of Justice (2016)


In [606]:
#movie_count[movie_count["movie"]=="Batman v Superman: Dawn of Justice (2016)"].loc[9913]["id"]

In [632]:
movie2_is_Batman = movie_pairs[movie_pairs["movie2"]==171798691881]
movie2_is_Batman.sort_values(by ="weight", ascending = False).head(5)

,movie1,movie2,weight
295108,146028888145,171798691881,0.029851
295091,103079215430,171798691881,0.025974
295097,111669150137,171798691881,0.025000
295083,85899345959,171798691881,0.023529
295092,103079215572,171798691881,0.015873


In [633]:
movie1_is_Batman = movie_pairs[movie_pairs["movie1"]==171798691881]
movie1_is_Batman.sort_values(by ="weight", ascending = False).head(5)

,movie1,movie2,weight
4092847,171798691881,635655159914,0.074627
5626433,171798691881,738734375161,0.067416
5827922,171798691881,755914244177,0.046875
26975822,171798691881,1632087572807,0.045455
21293505,171798691881,1451698946319,0.045455


In [634]:
Batman_neighbor_top = movie1_is_Batman.sort_values(by ="weight", ascending = False).head(5)
neighbors = np.array(Batman_neighbor_top["movie2"]).tolist()
id_member_movie = movie_id_membership[["id","membership","movie"]]

In [637]:
topfive = id_member_movie[id_member_movie["id"].isin(neighbors)]
topfive["movie"]=topfive["movie"].map(lambda x:movie_subname3(x))
topfive.merge(movie_genre1,left_on="movie",right_on="movie",how = "left")#.drop("membership",axis=1)

/Users/AlChen/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,id,membership,movie,genre
0,635655159914,6,Eloise (2015),Thriller
1,738734375161,6,Man of Steel (2013),Sci-Fi
2,755914244177,6,Demoted (2011),Comedy
3,1451698946319,6,Love and Honor (2013),War
4,1632087572807,6,Real Steel (2011),Sport


### Mission: Impossible - Rogue Nation (2015)

In [529]:
movie_count[movie_count["movie"]=="Mission: Impossible - Rogue Nation (2015)"]

,movie,count,id
90114,Mission: Impossible - Rogue Nation (2015),55,1563368096040


In [622]:
id_member_movie[id_member_movie["id"] == 1563368096040]

,id,membership,movie
90114,1563368096040,6,Mission: Impossible - Rogue Nation (2015)


In [607]:
movie2_is_Mission = movie_pairs[movie_pairs["movie2"]==1563368096040]
movie2_is_Mission.sort_values(by ="weight", ascending = False).head(5)

,movie1,movie2,weight
24744576,163208757380,1563368096040,0.129870
24744581,171798692139,1563368096040,0.098765
24744625,309237645603,1563368096040,0.094118
24744684,498216206651,1563368096040,0.089552
24744560,77309411587,1563368096040,0.088000


In [608]:
movie1_is_Mission = movie_pairs[movie_pairs["movie1"]==1563368096040]
movie1_is_Mission.sort_values(by ="weight", ascending = False).head(5)

,movie1,movie2,weight
26409378,1563368096040,1614907703603,0.056338
26627602,1563368096040,1623497638076,0.050633
28419827,1563368096040,1675037245673,0.042857
29717754,1563368096040,1709396984248,0.027778
26515122,1563368096040,1614907703787,0.023256


In [638]:
Mission_neighbor_top = movie2_is_Mission.sort_values(by ="weight", ascending = False).head(5)
neighbors = np.array(Mission_neighbor_top["movie1"]).tolist()
id_member_movie = movie_id_membership[["id","membership","movie"]]

In [639]:
topfive = id_member_movie[id_member_movie["id"].isin(neighbors)]
topfive["movie"]=topfive["movie"].map(lambda x:movie_subname3(x))
topfive.merge(movie_genre1,left_on="movie",right_on="movie",how = "left")#.drop("membership",axis=1)

/Users/AlChen/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,id,membership,movie,genre
0,77309411587,6,Muppets Most Wanted (2014),Musical
1,163208757380,6,Fan (2015),Drama
2,171798692139,6,Now You See Me: The Second Act (2016),Thriller
3,309237645603,6,Phantom (2015),Thriller
4,498216206651,6,Patient Zero (2015),Horror


### Minions (2015)

In [614]:
movie_count[movie_count["movie"]=="Minions (2015)  (voice)"]

,movie,count,id
79152,Minions (2015) (voice),18,1374389534996


In [623]:
id_member_movie[id_member_movie["id"] == 1374389534996]

,id,membership,movie
79152,1374389534996,6,Minions (2015) (voice)


In [ ]:
id_member_movie = movie_id_membership[["id","membership","movie"]]

In [615]:
movie2_is_Minions = movie_pairs[movie_pairs["movie2"]==1374389534996]
movie2_is_Minions.sort_values(by ="weight", ascending = False).head(5)

,movie1,movie2,weight
19080641,515396075930,1374389534996,0.303030
19080629,472446402736,1374389534996,0.275862
19080846,1348619731379,1374389534996,0.259259
19080807,1168231104898,1374389534996,0.257143
19080758,987842478223,1374389534996,0.235294


In [617]:
movie1_is_Minions = movie_pairs[movie_pairs["movie1"]==1374389534996]
movie1_is_Minions.sort_values(by ="weight", ascending = False).head(5)

,movie1,movie2,weight
22997043,1374389534996,1503238554059,0.233333
24922260,1374389534996,1571958030427,0.189189
25824225,1374389534996,1597727834358,0.166667
19615764,1374389534996,1391569404340,0.102564
26332678,1374389534996,1614907703467,0.096154


In [640]:
Minions_neighbor_top = movie2_is_Minions.sort_values(by ="weight", ascending = False).head(5)
neighbors = np.array(Minions_neighbor_top["movie1"]).tolist()
id_member_movie = movie_id_membership[["id","membership","movie"]]

In [641]:
topfive = id_member_movie[id_member_movie["id"].isin(neighbors)]
topfive["movie"]=topfive["movie"].map(lambda x:movie_subname3(x))
topfive.merge(movie_genre1,left_on="movie",right_on="movie",how = "left")#.drop("membership",axis=1)

/Users/AlChen/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,id,membership,movie,genre
0,472446402736,6,Inside Out (2015),Family
1,515396075930,6,The Lorax (2012),Fantasy
2,987842478223,6,Gake no ue no Ponyo (2008),Fantasy
3,1168231104898,6,Surf's Up (2007),Sport
4,1348619731379,6,Up (2009),Adventure
